# Defining Model Tester

In [1]:
from french_crs.fast_model_training import model_tester

train="balanced_Full_Soon"
test="balanced_Full_Soon"

model_parameter={
                "model_name" : "../pre-trained language models/Model_ANCOR_"+train+".model",
                "column_gold" : "IS_CO_REF",
                "column_outcome" : "Prediction",
                "columns_drop_list" : [ 
                                        "SUB_CORPUS",
                                        "FILE_NAME",
                                        "COREF_TABLE_ID",
                                        "COREF_TYPE",
                                        "COREF_ANCOR_ID",
                                        "MENTION_LEFT_ID",
                                        "MENTION_RIGHT_ID",
                                        "LEFT_CONTENT",
                                        "RIGHT_CONTENT",
                                        "LEFT_DEF",
                                        "RIGHT_DEF",
                                        "ID_DEF"
#                                         "DISTANCE_MENTION",
#                                         "DISTANCE_WORD",
#                                         "DISTANCE_CHAR"
                                       ],
                "threshold" : 0.5
                }

model=model_tester(**model_parameter)

# Looping over Test Files

In [7]:
import glob
import json
import pandas as pd

window_size=20
counter=0
acc=0
json_relations_positive={}
split_config_json="../datasets/default_split_config.json"

with open(split_config_json) as json_file:
    config_json = json.load(json_file)

sub_corpus_files_test = config_json["sub_corpus_files_test"]


for sub_corpus in ["corpus_OTG","corpus_UBS","corpus_ESLO","corpus_ESLO_CO2"]:

    ancor_subcorpus_path = "../DISTRIB_ANCOR_EN_CHAINE_Medium/"+sub_corpus+'/'

    corpus_files = [file[file.rfind(
        "/")+1:-4] for file in glob.glob(ancor_subcorpus_path+"annotation_integree/*.xml")]

    set1=set(corpus_files)
    set2=set(sub_corpus_files_test)
    corpus_files=set1.intersection(set2)
    
#     "1AG0141"

    list_dataframes = []

    for subfile in corpus_files:

        root_xml,root_aa,data_source=set_file_config(sub_corpus, subfile)
        json_mentions=generate_json_mentions(root_xml,root_aa)
        json_mentions=remove_empty_json_mentions(json_mentions)
        json_relations=generate_json_relations()
        if json_relations == {}:
            continue
        json_coreference_chains=generate_json_chains()
        
        df_mentions = pd.DataFrame.from_dict(json_mentions, orient='index')
        df_mentions = df_mentions.sort_values(by=['START_ID'])
        list_mentions_sorted = list(df_mentions.index)
        list_mentions_sorted_rev=list(reversed(list_mentions_sorted))
        
        print("Starting",subfile,"at",sub_corpus)
        
        for index, right_mention in enumerate(list_mentions_sorted_rev):
            
            left_mention_candidates=list_mentions_sorted_rev[index+1:index+window_size]
            
            if left_mention_candidates!=[]:
                for left_mention_candidate in left_mention_candidates:
                    
                    
                    coref_features=generate_coref_features(left_mention_candidate,right_mention,
                                                           json_relations,json_mentions,
                                                           data_source)
                    pred=model.apply_model_to_row(coref_features)
                    
                    
                    if pred>=0.7480658909510631:
#                         print(left_mention_candidate,right_mention,"\n\n")
                        counter+=1
                        acc+=pred
                        print(acc/counter)
                        json_relations_positive[counter]=coref_features
                        break

df_coref_relations = pd.DataFrame.from_dict(json_relations_positive, orient='index')
df_coref_relations.to_excel("Dataset_Test.xlsx")
print(acc,counter)

Starting 1PF0048 at corpus_OTG
0.8929225916532602
0.9422612958266301
0.9429540865634252
0.9395488982559022
0.9516391186047217
0.9336992655039348
0.9311707990033727
0.9372744491279512
Starting 2AP0307 at corpus_OTG
0.9429878701243164
0.9367194414812593
0.9391813104375085
0.9441133849541462
0.9444027178467505
0.9482616843551935
0.9502075546583303
0.9494134264470451
0.9500496114459585
0.9440098182174794
0.945104244484624
0.9478490322603929
0.9494650660557385
0.9495827136539308
0.9472187748521129
0.9489385452217843
0.9444743367462464
0.9429421594970668
0.9418265373423905
0.9417038097820265
0.9415697727340593
0.9435062839412183
0.9453282317710715
0.9462559659591869
0.9473613954590135
0.9476218573849745
0.9481453069207687
0.9450389314361275
0.9444490176462896
0.9450590847578522
0.9464678261743175
0.9446409661678757
0.940223205021602
0.9416320322905447
0.9393984717953846
0.9403794606537059
0.9417040652317717
0.9427634702933038
0.940860701421815
0.939659689334113
0.9374962083845805
0.936666284

0.9444849612209031
0.9446146287996714
0.9447500453551734
0.9445726062201608
0.9447001667889683
0.9446322141226396
0.9447623206372026
0.9446566438269971
0.9447842978757269
0.944769402637535
0.9447271099053907
0.9448230967943964
0.9449478398982939
0.9450695273173054
0.9451637166077421
0.9452936604072498
0.9453706428358732
0.9454994856593736
0.945623255620425
0.9455614387680046
0.9453366102881056
0.9454642973357189
0.9455817018886752
0.9453093860371139
0.9454321744784702
0.9454656521639476
0.9455915355380566
0.9457168699415788
0.9453293495519325
0.9453174976143407
0.945437642863393
0.9455573609084501
0.9456813760316655
0.9458047971467449
0.945926314500873
0.9460486531558484
0.9459537352029006
0.9460203272724301
0.946137440251354
0.9462542855336058
0.9461823683503655
0.9462944222536526
0.9464140337853817
0.9464936459280706
0.9466044015409426
0.9467183556797185
0.9467366711527683
0.9468539912603613
0.9467841152838704
0.9469008167854409
0.9470085431833682
0.9471242450541469
0.947233103535902

0.9500622470920452
0.9501044978240862
0.9501260586700813
0.9500131702594959
0.949861970178152
0.9498086752615218
Starting 1PF0305 at corpus_OTG
0.9495749652746507
0.9496263890109713
0.9495000416556951
0.9493241204861783
0.9492739042890489
0.9492588644091673
0.9493167360623176
0.9493770015366786
0.949361177377419
0.9494212471551444
0.9494385205589891
0.9494802071483885
0.9495198659396626
0.9495748977161663
0.9495810992185724
0.9495877473824997
0.9496470400167397
0.9496874077723018
0.9495868356974517
0.9495391315012444
0.9495841676470276
0.949619741719955
0.9496458868814971
0.9496639196119568
0.9496656082532636
0.9497064203723998
0.9497639703037829
0.9498007344253918
0.9497731529553852
0.9498035432764103
0.9498463373137427
0.9498626999295651
0.9498291990058726
0.9497763394914484
0.9497374266579329
0.9497199640373724
0.9497777571821571
0.9497290445917173
0.9496832157728287
0.9497381840582271
0.9497956766012574
0.9498521361501162
0.9499075703564899
0.9498689072203935
0.9499045231376138
0.9

0.9462534265639808
0.9462958223653635
0.9463383055850204
0.9463671847192238
0.9464093889899551
0.9463577003039688
0.9463508027874231
0.9462345848629288
0.9462206307615543
0.9462547063353132
0.9462111599863233
0.9462536808084685
0.9462959801666805
0.9463383669226657
0.9462712524909273
0.9461496257953075
0.9461416339639726
0.9461272030953934
0.9460409387147573
0.9460199618124545
0.9459122125146759
0.9457950161299365
0.9457459699740449
0.9456789332677628
0.9456771606964344
0.9456811182061664
0.945536892893174
0.9455793985713735
0.9456208001495162
0.9456617314065264
0.9457031396937892
0.9456957159557892
0.94573508660403
0.9457772504839026
0.9458165270410848
0.9458557125024305
0.9458963575805311
0.9459370484336561
0.9459600614783965
0.9460005989976064
0.9458841637072939
0.9459209425280227
0.94595467786213
0.9459838559468565
0.945879605415824
0.9458776110900448
0.9457546006509924
0.9457382734706892
0.9456215416433394
0.9455292252900702
0.9454574135119846
0.9454397960814519
0.9454214506824534

0.9448324529879543
0.9447409057043078
0.9446696482390826
0.9447014025598385
0.9446963565987296
0.9447290147854207
0.9447616344146305
0.9447927926996706
0.9448239706101813
0.9447728341835079
0.9447111305120216
0.9447426314383016
0.9446649396943183
0.9446810907768228
0.9446494798342172
0.9446810638848986
0.9446419584803566
0.944636853640083
0.9445688925524248
0.9446004498023323
0.9445625104289117
0.944494712141999
0.9445262522220125
0.9445586662573395
0.9445867834137869
0.9445672846812309
0.9445861739694149
0.9446184852382374
0.94465044425459
0.9446425727950871
0.9445566323974747
0.9445802109149596
0.9446124317225671
0.9446236970150003
0.9446178740644483
0.9446490686846298
0.9446811669548165
0.944712131446803
0.9446900202477222
0.9447192548764781
0.9447111225916345
0.944708397824375
0.9446167797278923
0.9445817613687195
0.9444921512576391
0.9444960405017204
0.9445249705669905
0.9445315844168077
0.9444212926335722
0.9443865077788608
0.944381682400392
0.9444046516427702
0.9443112268168522


0.9444357014073228
0.9444468009356284
0.9444726705916461
0.9444705658054854
0.9444741324787734
0.9444550097084496
0.9444506979738448
0.944461759177239
0.9444856293907697
0.944416261796867
0.9444204997680727
0.9444096759675876
0.9443926142637602
0.9444181118553238
0.944440512949689
0.9444661363792177
0.9444801878776473
0.9444949236476905
0.9444488243539725
0.9444449545791055
0.9444316749551189
0.9444321150936256
0.9444268333622855
0.9443926097026759
0.944322218613967
0.9443133161651677
0.9442647639338096
0.9442892028650315
0.9442847773638439
0.9443058826682312
0.9442383850709453
0.9442509547174723
0.9442731223336667
0.944284132485716
0.9442973502590732
0.9442768994197288
0.9443011406332388
0.9443250863763453
0.9443505812207623
0.9443189520762826
0.9443376129164885
0.9443630666092304
0.9443887768372299
0.9443903524599377
0.9444123329066322
0.9444379848065368
0.9444636068922657
0.9444892053522815
0.9445062198467308
0.9445278571532938
0.9445500255471435
0.9445438159395617
0.944547858955842

0.9433439289880979
0.9433653299011555
0.9433363998516957
0.94331765480914
0.9432455635017031
0.9431772508584271
0.9431989647567468
0.9432138290157593
0.9431535485864682
0.9431402957074487
0.9431443962675992
0.9431664018556004
0.9431614598618085
0.9431156123417586
0.9431376074785174
0.9431557041657733
0.9431435954583633
0.9431537379881851
0.9431758090284487
0.9431240193175061
0.9430746890810615
0.9430548773319998
0.9430376659280969
0.9430364110188224
0.9429756487870867
0.9429736649142678
0.9429029157491554
0.9429247776046384
0.942934477884095
0.9429063694637152
0.9429099357705127
0.9429070911319857
0.942896431804475
0.9428853763547629
0.9428523574265416
0.9427813078351207
0.9427849512071196
0.9427823273923576
0.9427370676643965
0.9427591258047414
0.9427808586640241
0.942802649692374
0.9428181865467574
0.9428339661215273
0.9428371949739414
0.9428461395817014
0.9428361117805893
0.9428419438940304
0.9428377139237373
0.9428228107334365
0.9428202563896046
0.9428238638723649
0.942830534898641

0.9424991993961926
0.9425184047404157
0.9425370066768168
0.9425561865811303
0.9425597825593592
0.9425684260661349
0.9425873742236045
0.9426038450988634
0.9426116073079448
0.9426152182370253
0.9425769583005421
0.9425840232278722
0.9426031300421058
0.942603681421368
0.9426226541732248
0.9426406735782428
0.9426586262960688
0.94259777625411
0.9425973563129207
0.9426164142955525
0.9426234450242962
0.9426424817047792
0.9426609844468223
0.9426612337652737
0.942673619359461
0.942681018254416
0.9426841226913029
0.9426223398726012
0.9426200815674465
0.9426205381916797
0.9426395191582058
0.9426583733999704
0.9426077018877645
0.942621610524978
0.9425698179395866
0.9425887799107646
0.9426077293616579
0.942626670705103
0.9426450787220637
0.9426473015964665
0.9426656754086065
0.9426307047299771
0.9426496029052446
0.9425907591625222
0.9426096624357647
0.942628553264456
0.9426468940513506
0.9426598792176619
0.9426750174481379
0.9426936844895092
0.9427059687736083
0.9426826750913568
0.9426706787102609
0

In [10]:
from french_crs.fast_pairs2chains import chains_builder

model_chains=chains_builder(path_gold_file="Dataset_Test.xlsx", 
                            path_model_file="Dataset_Test.xlsx",
                            gold_column="IS_CO_REF",
                            model_column="IS_CO_REF",
                            scorch_output_path="./",
                            threshold=0.5)

model_chains.generate_gold_model_json_output(mode="train")

SCORCH json formats saved at following addresses:
/Users/mehdi.mirzapour/French-CRS/demo/coref_chains_gold.json
/Users/mehdi.mirzapour/French-CRS/demo/coref_chains_pred.json


In [ ]:
import joblib
import numpy as np

model1 = joblib.load(model.model_name)

df_coref_relations = pd.DataFrame.from_dict(coref_features, orient='index')

input_dataframe = df_coref_relations.drop(model.columns_drop_list, axis=0)

input_dataframe.fillna("VOID", inplace=True)
input_dataframe = input_dataframe.replace(
                        model.values_list, model.values_list_for_replace)

y_pred = model1.predict(np.array(input_dataframe[0:-1]).reshape(1, -1))[0]
y_pred

In [ ]:
json_relations_positive

In [ ]:
y_pred

In [3]:
def generate_coref_features(pair_left_id,pair_right_id,
                            json_relations,json_mentions,
                            data_source):

    relation_type="CO_REFERENCE"
    relation_left=json_mentions[pair_left_id]
    relation_right=json_mentions[pair_right_id]

    if relation_left["START_ID"] > relation_right["START_ID"]:
        relation_left,relation_right=relation_right,relation_left

    MENTION_LEFT_ID=pair_left_id
    MENTION_RIGHT_ID=pair_right_id

    string_text = data_source[relation_left["END_ID"]:relation_right["START_ID"]]
    string_text=re.sub("<[a-zA-Z0-9\s=\",/.]+>", " ", string_text).strip().split()
    DISTANCE_MENTION = int(relation_right["NUM"])-int(relation_left["NUM"])-1
    DISTANCE_WORD = len(string_text)
    DISTANCE_CHAR = len("".join(string_text))


    tf2yn = {True: "YES", False: "NO"}

    left_content = str(relation_left["CONTENT"])
    right_content = str(relation_right["CONTENT"])
    left_content_list = left_content.lower().split()
    right_content_list = right_content.lower().split()
    len_of_words_min = min(len(left_content_list), len(right_content_list))
    len_of_words_max = max(len(left_content_list), len(right_content_list))
    len_intersection = len(
        [value for value in left_content_list if value in right_content_list])


    ID_FORM = tf2yn[relation_left["CONTENT"]==relation_right["CONTENT"]]
    ID_SUBFORM = tf2yn[len_intersection>0]

    INCL_RATE = len_intersection/len_of_words_min
    COM_RATE = len_intersection/len_of_words_max

    ID_DEF = tf2yn[relation_left["DEF"]==relation_right["DEF"]]
    if relation_left["DEF"] == "UNK" or relation_right["DEF"] == "UNK":
        ID_DEF = "UNK"

    ID_GP = tf2yn[relation_left["GP"]==relation_right["GP"]]
    if relation_left["GP"] == "UNK" or relation_right["GP"] == "UNK":
        ID_GP = "UNK"

    ID_TYPE = tf2yn[relation_left["TYPE"]==relation_right["TYPE"]]
    if relation_left["TYPE"] == "UNK" or relation_right["TYPE"] == "UNK":
        ID_TYPE = "UNK"

    ID_EN = tf2yn[relation_left["EN"]==relation_right["EN"]]
    if relation_left["EN"] == "UNK" or relation_right["EN"] == "UNK":
        ID_EN = "UNK"

    ID_GENDER = tf2yn[relation_left["GENRE"]==relation_right["GENRE"]]
    if relation_left["GENRE"] == "UNK" or relation_right["GENRE"] == "UNK":
        ID_GENDER = "UNK"

    ID_NUMBER = tf2yn[relation_left["NUM"]==relation_right["NUM"]]
    if relation_left["NUM"] == "UNK" or relation_right["NUM"] == "UNK":
        ID_NUMBER = "UNK"

    EMBEDDED = tf2yn[((left_content in right_content) or (right_content in left_content))]

    ID_PREVIOUS = tf2yn[relation_left["PREVIOUS"]==relation_right["PREVIOUS"]]
    if relation_left["PREVIOUS"] == "^" or relation_right["PREVIOUS"] == "^":
        ID_PREVIOUS = "UNK"

    ID_NEXT = tf2yn[relation_left["NEXT"]==relation_right["NEXT"]]
    if relation_left["NEXT"] == "^" or relation_right["NEXT"] == "^":
        ID_NEXT = "UNK"
        
    coref_relation_json_feature={
        
                                "SUB_CORPUS":sub_corpus,
                                "FILE_NAME":subfile,
                                "COREF_TABLE_ID":-1,
                                "COREF_TYPE":relation_type,
                                "COREF_ANCOR_ID":"",
                                "MENTION_LEFT_ID":MENTION_LEFT_ID,
                                "MENTION_RIGHT_ID":MENTION_RIGHT_ID,
                                "LEFT_CONTENT":relation_left["CONTENT"],                                     
                                "RIGHT_CONTENT":relation_right["CONTENT"],                                     
                                "LEFT_TYPE":relation_left["TYPE"], 
                                "RIGHT_TYPE":relation_right["TYPE"], 
                                "LEFT_DEF":relation_left["DEF"],
                                "RIGHT_DEF":relation_right["DEF"], 
                                "LEFT_GP":relation_left["GP"],
                                "RIGHT_GP":relation_right["GP"],
                                "LEFT_GENRE":relation_left["GENRE"],
                                "RIGHT_GENRE":relation_right["GENRE"],
                                "LEFT_NB":relation_left["NB"],
                                "RIGHT_NB":relation_right["NB"],
                                "LEFT_EN":relation_left["EN"],
                                "RIGHT_EN":relation_right["EN"],
                                "ID_FORM": ID_FORM, 
                                "ID_SUBFORM": ID_SUBFORM, 
                                "INCL_RATE": INCL_RATE, 
                                "COM_RATE": COM_RATE, 
                                "ID_DEF": ID_DEF, 
                                "ID_GP": ID_GP, 
                                "ID_TYPE": ID_TYPE,
                                "ID_EN": ID_EN, 
                                "ID_GENDER": ID_GENDER, 
                                "ID_NUMBER": ID_NUMBER, 
                                "DISTANCE_MENTION": DISTANCE_MENTION, 
                                "DISTANCE_WORD": DISTANCE_WORD,
                                "DISTANCE_CHAR": DISTANCE_CHAR, 
                                "EMBEDDED": EMBEDDED, 
                                "ID_PREVIOUS": ID_PREVIOUS, 
                                "ID_NEXT": ID_NEXT, 
                                "IS_CO_REF": 1
    }
    
    return(coref_relation_json_feature)
                    

In [6]:
import xml.etree.ElementTree as ET
import json
import pandas as pd
import re
import random
import glob
import networkx as nx
import os


def set_file_config(sub_corpus, file):

    path_file = "../DISTRIB_ANCOR_EN_CHAINE_Medium/"+sub_corpus+'/'

    tree_xml = ET.parse(
        path_file+'annotation_integree/'+file+'.xml')
    tree_aa = ET.parse(
        path_file+'aa_fichiers/'+file+'.aa')
    data_source = open(
        path_file+'ac_fichiers/'+file + '.ac', "r").read()

    root_xml = tree_xml.getroot()
    root_aa = tree_aa.getroot()
    
    return(root_xml,root_aa,data_source)
    

# return mentions in json format
def generate_json_mentions(root_xml,root_aa):

    json_mentions = {}

    for unit in root_xml.iter("unit"):
        data_character = {}
        data_character["TYPE"] = unit.find("./characterisation/type").text

        for anchor in root_xml.iter("anchor"):
            if anchor.attrib["id"] == unit.attrib["id"]:
                data_character["NUM"] = anchor.attrib["num"]

        for feat in unit.findall("./characterisation/featureSet/feature"):
            data_character[feat.attrib["name"]] = feat.text

        for unit_aa in root_aa.iter("unit"):
            if unit_aa.attrib["id"] == unit.attrib["id"]:
                data_character["START_ID"] = int(unit_aa.find(
                    "./positioning/start/singlePosition").attrib["index"])
                data_character["END_ID"] = int(unit_aa.find(
                    "./positioning/end/singlePosition").attrib["index"])

        json_mentions[unit.attrib["id"]] = data_character

    return(json_mentions)

# return mentions in json format and eliminates all the features that are not presented.
def remove_empty_json_mentions(json_mentions,features_to_check=["CONTENT", "PREVIOUS"]):

    json_mentions_= json_mentions.copy()
    for mention in list(json_mentions_):
        for feature in features_to_check:
            if feature not in json_mentions_[mention]:
                del json_mentions_[mention]
                break

    return(json_mentions_)


# return relations in json format based on the json mentions
def generate_json_relations():

    json_relations={}

    for relation in root_aa.iter("relation"):
        data_character = {}
        data_character["TYPE"] = relation.find("./characterisation/type").text

        for feat in relation.findall("./characterisation/featureSet/feature"):
            data_character[feat.attrib["name"]] = feat.text

        unit_ids = []
        for feat in relation.findall("./positioning/term"):
            unit_ids.append(feat.attrib["id"])

        if (unit_ids[0] in json_mentions.keys()) and (unit_ids[1] in json_mentions.keys()):
            if (len(unit_ids) == 2):
                data_character["LEFT_UNIT"] = {
                    "ID": unit_ids[0], **json_mentions[unit_ids[0]]}
                data_character["RIGHT_UNIT"] = {
                    "ID": unit_ids[1], **json_mentions[unit_ids[1]]}

            json_relations[relation.attrib["id"]] = data_character

    return(json_relations)


# return chains in json format based on the json mentions and relations.
def generate_json_chains(coreference_type=["DIRECTE", "INDIRECTE", "ANAPHORE"]):
    G = nx.Graph()
    chain_id = 0
    json_coreference_chains = {}

    for chain in json_relations:
        if json_relations[chain]["TYPE"].strip() in coreference_type:
            chain_left = json_relations[chain]["LEFT_UNIT"]["ID"]
            chain_right = json_relations[chain]["RIGHT_UNIT"]["ID"]
            G.add_edge(chain_left, chain_right,
                       coref_type=json_relations[chain]["TYPE"].strip())

    for subgraph in list(nx.connected_components(G)):
        json_coreference_chains[chain_id] = []
        for node in subgraph:
            json_coreference_chains[chain_id].append(node)
        chain_id += 1

    return(json_coreference_chains)

# Dataset Splitting

In [ ]:
# Two ways of Training: (1) commented codes down with parameter.
#                       (2) uncommented with mentioned explicity the config file.

from french_crs.fast_model_training import dataset_splitter

# ds1=dataset_splitter("./Dataset_ANCOR_Balanced.xlsx",
#                     "./Dataset_ANCOR_Balanced_Train.xlsx",
#                     "./Dataset_ANCOR_Balanced_Test.xlsx",
#                       split_config_json="./split_config.json")

# ds1.dataset_splitter_by_file(lower_rate=0.20,upper_rate=0.50, files_num=68)


ds2=dataset_splitter("./Dataset_ANCOR_Balanced.xlsx",
                    "./Dataset_ANCOR_Balanced_Train.xlsx",
                    "./Dataset_ANCOR_Balanced_Test.xlsx",
                      split_config_json="./split_config.json")

dict_files=ds2.dataset_splitter_by_json_config()


# Model Training

In [ ]:
from french_crs.model_training import model_trainer

"""
12 possibilities for "train" and "test" variables

balanced_ANAPHORE
balanced_DIRECTE
balanced_Full
balanced_INDIRECTE
representative_ANAPHORE
representative_DIRECTE
representative_Full
representative_INDIRECTE
window_ANAPHORE
window_DIRECTE
window_Full
window_INDIRECTE
"""

train="balanced_Full_Soon"
test="balanced_Full_Soon"

model=model_trainer( "../datasets/"+ train+"/"+train+"_Train.xlsx",
                     "../datasets/"+ test+"/"+test+"_Test.xlsx",
                     "../datasets/"+ train+"/"+train+"_Test_Pred.xlsx",
                     "IS_CO_REF",
                     "IS_CO_REF"
                   )

model.columns_drop_list = ["SUB_CORPUS",
                            "FILE_NAME",
                            "COREF_TABLE_ID",
                            "COREF_TYPE",
                            "COREF_ANCOR_ID",
                            "MENTION_LEFT_ID",
                            "MENTION_RIGHT_ID",
                            "LEFT_CONTENT",
                            "RIGHT_CONTENT",
                            "LEFT_DEF",
                            "RIGHT_DEF",
                            "ID_DEF",
                            "DISTANCE_MENTION",
                            "DISTANCE_WORD",
                            "DISTANCE_CHAR"]

model.convert_columns_to_numeric()
print("Model_Name :","Model_ANCOR_"+train+".model\n")
print("Train Dataset :",train+"_Train.xlsx")
print("Test  Dataset :",test+"_Test.xlsx")
performance=model.train_model_random_forest(model_name="../pre-trained language models/Model_ANCOR_"+train+".model",max_depth=10, random_state=0,n_estimators=250)
performance

In [ ]:
train="balanced_Full_Soon"
test="balanced_Full_Soon"

# SCORCH Chains Building

In [ ]:
from french_crs.fast_pairs2chains import chains_builder

model_chains=chains_builder(path_gold_file="../datasets/"+ train+"/"+train+"_Test_Pred.xlsx", 
                            path_model_file="../datasets/"+ train+"/"+train+"_Test_Pred.xlsx",
                            gold_column="IS_CO_REF",
                            model_column="Prediction",
                            scorch_output_path="../",
                            threshold=0.5)

model_chains.generate_gold_model_json_output(mode="train")

# SCORCH Outcome

In [ ]:
import os
bashCommand = "scorch ../coref_chains_gold.json ../coref_chains_pred.json > ../mm.txt"
os.system(bashCommand)
f = open("../mm.txt",'r')
message = f.read()
print(message)

# Putting All Chains Together

In [ ]:
import docx
import os
from french_crs.model_training import model_trainer
from french_crs.pairs2chains import chains_builder

mydoc = docx.Document()

style = mydoc.styles['Normal']
font = style.font
font.name = 'MS Gothic'
font.size = docx.shared.Pt(10)


train_test_list=[
    "balanced_ANAPHORE",
    "balanced_DIRECTE",
    "balanced_Full",
    "balanced_INDIRECTE",
    "representative_ANAPHORE",
    "representative_DIRECTE",
    "representative_Full",
    "representative_INDIRECTE",
    "window_ANAPHORE",
    "window_DIRECTE",
    "window_Full",
    "window_INDIRECTE"
    ]

# train_test_list=[
#     ["balanced_ANAPHORE","window_ANAPHORE"],
#     ["balanced_DIRECTE","window_DIRECTE"],
#     ["balanced_Full","window_Full"],
#     ["balanced_INDIRECTE","window_INDIRECTE"],
#     ["representative_ANAPHORE","window_ANAPHORE"],
#     ["representative_DIRECTE","window_DIRECTE"],
#     ["representative_Full","window_Full"],
#     ["representative_INDIRECTE","window_INDIRECTE"],
#     ["window_ANAPHORE","window_ANAPHORE"],
#     ["window_DIRECTE","window_DIRECTE"],
#     ["window_Full","window_Full"],
#     ["window_INDIRECTE","window_INDIRECTE"]
#     ]

counter=0

# for train in train_test_list:
#     for test in train_test_list:

for train in train_test_list:
    
    test=train
    
    counter+=1

    model=model_trainer( "../datasets/"+ train+"/"+train+"_Train.xlsx",
                 "../datasets/"+ test+"/"+test+"_Test.xlsx",
                 "../datasets/"+ train+"/"+train+"_Test_Pred.xlsx",
                 "IS_CO_REF",
                 "IS_CO_REF"
               )

    model.columns_drop_list = ['m1_DEF', 'm2_DEF', 'ID_DEF',
                               'DISTANCE_MENTION','DISTANCE_WORD',
                               'DISTANCE_CHAR']

    model.convert_columns_to_numeric()
    performance=model.train_model_random_forest(model_name="../pre-trained language models/Model_ANCOR_"+train+".model",max_depth=10, random_state=0,n_estimators=250)


    model_chains=chains_builder(path_gold_file="../datasets/"+ train+"/"+train+"_Test_Pred.xlsx", 
                        path_model_file="../datasets/"+ train+"/"+train+"_Test_Pred.xlsx",
                        gold_column="IS_CO_REF",
                        model_column="Prediction",
                        scorch_output_path="../",
                        threshold=0.5)

    model_chains.generate_gold_model_json_output(mode="train")


    bashCommand = "scorch ../coref_chains_gold.json ../coref_chains_pred.json > ../output.txt"
    os.system(bashCommand)
    f = open("../output.txt",'r')
    message = f.read()
    f.close()


    mydoc.add_paragraph("Model_Name : "+"Model_ANCOR_"+train+".model")
    mydoc.add_paragraph("Train Dataset : "+train+"_Train.xlsx")
    mydoc.add_paragraph("Test  Dataset : "+test+"_Test.xlsx")
    mydoc.add_paragraph("\n")
    mydoc.add_paragraph(str(performance))
    mydoc.add_paragraph("\n")
    mydoc.add_paragraph(message)
    mydoc.add_page_break()

    print(counter)


mydoc.save("../pre-trained language models/Performance Analysis.docx")

#  Model Testing

In [ ]:
from model_training import model_tester

model_parameter={
                "model_name" : "./Models/Random_Forest_(Normal)_OTG_Neg_90_Pos_10.model",
                "input_file" : "./Datasets/corpus_ALL_Window_30_Test.xlsx",
                "output_file" : "./Datasets/corpus_ALL_Window_30_Test_Called_Seperately.xlsx",
                "column_gold" : "IS_CO_REF",
                "column_outcome" : "Prediction",
                "threshold" : 0.5
                }

model=model_tester(**model_parameter)
model.apply_model_to_dataset()